| Action | Time | Notes|
|  ----  |  --- |  --- |
| Open Remote Dataset (fsspec + xarray) | 32.9s | Use `simple_templates=True` |
| Plotting + animation | 1min 46s | |

***

In [ ]:
from IPython.display import HTML

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import os

from azure.storage.blob import ContainerClient
import tempfile
from metpy import xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pandas as pd

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
%%time
fs = fsspec.filesystem('reference', fo='zip://combined.json::combined.zip', remote_protocol='az', remote_options={'account_name':'goeseuwest'}, simple_templates=True)

m = fs.get_mapper("")

ds = xr.open_dataset(m, engine='zarr', chunks='auto')

In [ ]:
ds

## Lat/lon

In [ ]:
def calc_latlon(ds):
    # The math for this function was taken from 
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm 

    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection
    
    x,y = np.meshgrid(x,y)
    
    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat
    
    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2
    
    r_s = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    
    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)
    
    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)
    
    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"

    return ds

In [ ]:
def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return ((min(x), max(x)), (min(y), max(y)))


In [ ]:
ds = calc_latlon(ds)

In [ ]:
top = 49.3457868 # north lat
left = -124.7844079 # west long
right = -66.9513812 # east long
bottom =  24.7433195 # south lat

lats = (bottom, top)
lons = (left, right)

((x1, x2), (y1,y2)) = get_xy_from_latlon(ds, lats, lons)

In [ ]:
%%time
subset = ds.sel(x=slice(x1, x2), y=slice(y2,y1))
subset

### Calculate true color
https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html

In [ ]:
nt = len(subset.t)

from matplotlib.animation import FuncAnimation

fig = plt.figure(figsize=(7.5,7.5), dpi=100)

dummy_channel = subset.metpy.parse_cf('CMI_C01')
x = dummy_channel.x; y = dummy_channel.y
ax = fig.add_subplot(111, projection = dummy_channel.metpy.cartopy_crs)

p = ax.imshow(subset.CMI_C13.isel(t=0), origin='upper', extent=(x.min(), x.max(), y.min(), y.max()),
         vmin=220, vmax=290)
ax.coastlines()
ax.add_feature(ccrs.cartopy.feature.BORDERS)
ax.add_feature(ccrs.cartopy.feature.STATES)

ts = pd.to_datetime(str(subset.t[0].values)).strftime("%Y-%m-%d %H%M UTC")
ax.set_title(f"GOES 16 Clean IR {ts}")
plt.colorbar(p, orientation='horizontal', label='Brightness Temp [K]', ax=ax)


def update_anim(i):
    ts = pd.to_datetime(str(subset.t[i].values)).strftime("%Y-%m-%d %H%M UTC")
    
    ax.clear()
    
    p = ax.imshow(subset.CMI_C13.isel(t=i), origin='upper', extent=(x.min(), x.max(), y.min(), y.max()),
                 vmin=220, vmax=290)
    ax.set_title(f"GOES 16 Clean IR {ts}")
    ax.coastlines()
    ax.add_feature(ccrs.cartopy.feature.BORDERS)
    ax.add_feature(ccrs.cartopy.feature.STATES)
#     plt.colorbar(p, orientation='horizontal', label='Brightness Temperature [K]', ax=ax)
    

    return p
  
# update_anim(1)
animator = FuncAnimation(fig, update_anim, frames=nt, interval=50)
# plt.show()

In [ ]:
%time HTML(animator.to_html5_video())